In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from tqdm import tqdm

# Zusätzliche Importe für Analyse und Visualisierung
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Funktion zur Datenbereinigung
def clean_data(df, is_train=True):
    """
    Bereinigt den DataFrame:
    - Ersetzt fehlende Werte (NaN) mit sinnvollen Standardwerten.
    - Entfernt problematische Spaltennamen.
    - Wandelt kategorische Spalten in numerische Werte um.
    """
    df.fillna(0, inplace=True)
    df.columns = df.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).astype('category').cat.codes
    return df

# Hauptskript
def main():
    # Aktuelles Arbeitsverzeichnis abrufen
    current_dir = os.getcwd().replace("\\", "/")

    # Einlesen der Trainings- und Testdaten
    train_path = os.path.join(current_dir, "data/train.csv")
    test_path = os.path.join(current_dir, "data/test.csv")
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Daten bereinigen
    df_train = clean_data(df_train, is_train=True)
    df_test = clean_data(df_test, is_train=False)

    # Abstimmen der Columns in df_test auf die von df_train
    missing_cols = set(df_train.columns) - set(df_test.columns)
    for col in missing_cols:
        df_test[col] = 0
    extra_cols = set(df_test.columns) - set(df_train.columns)
    df_test = df_test.drop(columns=extra_cols)
    df_test = df_test[df_train.columns]

    # ---------------------------
    # Analyse: Einfluss von "race_group" und "age_at_hct" auf den EFS Score
    # ---------------------------
    print("Analysiere Zusammenhänge zwischen 'race_group', 'age_at_hct' und EFS Score...")

    # Stelle sicher, dass "race_group" als kategorial interpretiert wird
    df_train['race_group'] = df_train['race_group'].astype('category')

    # OLS-Regression durchführen
    model = smf.ols("efs ~ age_at_hct + C(race_group)", data=df_train).fit()
    print(model.summary())

if __name__ == "__main__":
    main()


In [ ]:
    # ---------------------------
    # Grafik 1: Race Group vs. EFS
    # ---------------------------
    sns.set(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=(10, 6))
    # Boxplot zeigt die Verteilung des EFS Score für jede Race Group
    sns.boxplot(data=df_train, x="race_group", y="efs", palette="Set2")
    # Overlay mit Swarmplot für einzelne Datenpunkte
    sns.swarmplot(data=df_train, x="race_group", y="efs", color="0.25")
    plt.title("Race Group vs. EFS Score", fontsize=16)
    plt.xlabel("Race Group", fontsize=14)
    plt.ylabel("EFS Score", fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
    # ---------------------------
    # Grafik 2: Age at HCT vs. EFS
    # ---------------------------
    plt.figure(figsize=(10, 6))
    # Scatterplot mit Regressionslinie für den Zusammenhang mit age_at_hct
    sns.regplot(data=df_train, x="age_at_hct", y="efs", ci=None, scatter=True,
                marker="o", color="blue")
    plt.title("Age at HCT vs. EFS Score", fontsize=16)
    plt.xlabel("Age at HCT", fontsize=14)
    plt.ylabel("EFS Score", fontsize=14)
    plt.tight_layout()
    plt.show()